In [1]:
import numpy as np

np.set_printoptions(threshold=10000, suppress=True)
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
import nltk

# 1. Téléchargement du dataset

In [12]:
nltk.download('reuters')
nltk.download('punkt')

[nltk_data] Downloading package reuters to C:\Users\Bastien
[nltk_data]     Audu\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Bastien
[nltk_data]     Audu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
from nltk.corpus import reuters

train_documents, train_categories = zip(
    *[(reuters.raw(i), reuters.categories(i)) for i in reuters.fileids() if i.startswith('training/')])
test_documents, test_categories = zip(
    *[(reuters.raw(i), reuters.categories(i)) for i in reuters.fileids() if i.startswith('test/')])

In [4]:
print('Taille du corpus : {0:d}'.format(len(train_documents)))

Taille du corpus : 7769


# 2. Transformation des catégories en vecteurs

Les données textuelles ne sont pas directement utilisables par les modèles de machine learning. Il faut donc les transformer en vecteurs de nombres. Pour cela, on utilise la MultiLabelBinarizer de scikit-learn qui transforme les catégories en vecteurs de 0 et 1, chaque colonne correspondant à une catégorie.

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(train_categories)
test_labels = mlb.transform(test_categories)

# 3. Models

In [120]:
from sklearn.metrics import roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

algos = {
    'KNN5': KNeighborsClassifier(n_neighbors=5, n_jobs=-1, metric='cosine'),
    'MLP': MLPClassifier(max_iter=200, random_state=1, alpha=0.001),
    'OVSR': OneVsRestClassifier(LinearSVC(random_state=0)),
}


def run_models(X_train, Y_train, X_test, Y_test, algos):
    for algo_name in algos:
        model = algos[algo_name]
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        scores = roc_auc_score(Y_test, Y_pred)
        print('################## {0} #############'.format(algo_name))
        print('Aire sous la courbe: {:.3f}%'.format(scores.mean() * 100))
        print()


# 4. TF-IDF

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.porter import PorterStemmer

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

CV = CountVectorizer(max_features=1000, stop_words='english')
CV.fit(train_documents)
corpus_train_CV = CV.transform(train_documents)
corpus_test_CV = CV.transform(test_documents)

TFIDF = TfidfVectorizer(stop_words='english', tokenizer=tokenize)
TFIDF.fit(train_documents)
corpus_train_tfidf = TFIDF.transform(train_documents)
corpus_test_tfidf = TFIDF.transform(test_documents)

In [111]:
from sklearn.model_selection import GridSearchCV


def get_best_model2(models: dict, x_train, y_train, x_test, y_test):
    best_score = 0
    best_model = None
    for param in models:
        print('################## {0} #############'.format(param))
        model = models[param]
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        scores = roc_auc_score(y_test, y_pred)
        print('Aire sous la courbe: {:.3f}%'.format(scores.mean() * 100))
        if scores > best_score:
            best_score = scores
            best_model = model
    return best_model, best_score, best_model.get_params()

def get_best_model(model, X, y, cv=2, params=None):
    grid = GridSearchCV(model, param_grid=params, cv=cv)
    grid.fit(X, y)
    return grid.best_estimator_, grid.best_params_, grid.best_score_, grid

# 5. Exécution des modèles

In [112]:
knns = {
    'knn1': KNeighborsClassifier(n_neighbors=1, n_jobs=-1),
    'knn2': KNeighborsClassifier(n_neighbors=2, n_jobs=-1),
    'knn3': KNeighborsClassifier(n_neighbors=3, n_jobs=-1),
    'knn5': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    'knn7': KNeighborsClassifier(n_neighbors=7, n_jobs=-1),
    'knn9': KNeighborsClassifier(n_neighbors=9, n_jobs=-1),
}
knn_best_model, knn_best_score, knn_best_parameters = get_best_model2(knns, corpus_train_tfidf, train_labels, corpus_test_tfidf, test_labels)
print('Meilleur modèle KNN : {0}'.format(knn_best_model))
print('Meilleur paramètre KNN : {0}'.format(knn_best_parameters))
print('Meilleur score KNN : {0}'.format(knn_best_score))

################## knn1 #############
Aire sous la courbe: 73.573%
################## knn2 #############
Aire sous la courbe: 66.886%
################## knn3 #############
Aire sous la courbe: 70.753%
################## knn5 #############
Aire sous la courbe: 68.554%
################## knn7 #############
Aire sous la courbe: 66.638%
################## knn9 #############
Aire sous la courbe: 65.204%
Meilleur modèle KNN : KNeighborsClassifier(n_jobs=-1, n_neighbors=1)
Meilleur paramètre KNN : {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': -1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
Meilleur score KNN : 0.7357345230752118


In [114]:
mlps = {
    'mlp0.001': MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=200, random_state=1, alpha=0.001),
    'mlp0.01': MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=200, random_state=1, alpha=0.01),
    'mlp0.1': MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=200, random_state=1, alpha=0.1),
}

best_mlp_model,best_mlp_score, best_mlp_param = get_best_model2(mlps, corpus_train_tfidf, train_labels, corpus_test_tfidf, test_labels)
print('Meilleur modèle MLP : {0}'.format(best_mlp_model))
print('Meilleur paramètre MLP : {0}'.format(best_mlp_param))
print('Meilleur score MLP : {0}'.format(best_mlp_score))


################## mlp0.001 #############
Aire sous la courbe: 59.988%
################## mlp0.01 #############
Aire sous la courbe: 60.669%
################## mlp0.1 #############
Aire sous la courbe: 58.471%
Meilleur modèle MLP : MLPClassifier(alpha=0.01, hidden_layer_sizes=(20, 10), random_state=1)
Meilleur paramètre MLP : {'activation': 'relu', 'alpha': 0.01, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 1, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Meilleur score MLP : 0.6066935005179654


In [119]:
ovrs = {
    'ovr0.1': OneVsRestClassifier(LinearSVC(random_state=0, C=0.1)),
    'ovr1': OneVsRestClassifier(LinearSVC(random_state=0, C=1)),
    'ovr10': OneVsRestClassifier(LinearSVC(random_state=0, C=10)),
    'ovr100': OneVsRestClassifier(LinearSVC(random_state=0, C=100)),
    'ovr1000': OneVsRestClassifier(LinearSVC(random_state=0, C=1000)),
}

best_ovr_model,best_ovr_score, best_ovr_param = get_best_model2(ovrs, corpus_train_tfidf, train_labels, corpus_test_tfidf, test_labels)
print('Meilleur modèle OVR : {0}'.format(best_ovr_model))
print('Meilleur paramètre OVR : {0}'.format(best_ovr_param))
print('Meilleur score OVR : {0}'.format(best_ovr_score))

################## ovr0.1 #############
Aire sous la courbe: 57.656%
################## ovr1 #############
Aire sous la courbe: 68.476%
################## ovr10 #############
Aire sous la courbe: 71.530%
################## ovr100 #############
Aire sous la courbe: 72.174%
################## ovr1000 #############
Aire sous la courbe: 72.169%
Meilleur modèle OVR : OneVsRestClassifier(estimator=LinearSVC(C=100, random_state=0))
Meilleur paramètre OVR : {'estimator__C': 100, 'estimator__class_weight': None, 'estimator__dual': True, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__loss': 'squared_hinge', 'estimator__max_iter': 1000, 'estimator__multi_class': 'ovr', 'estimator__penalty': 'l2', 'estimator__random_state': 0, 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator': LinearSVC(C=100, random_state=0), 'n_jobs': None, 'verbose': 0}
Meilleur score OVR : 0.7217420965155602


On constate que le meilleur modèle va être le OneVsRestClassifier avec un C=1000, il est équivalent au MLP mais avec un temps de calcul plus rapide.

# 6. Vectorisation par SVD

In [121]:
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=100)
SVD.fit(corpus_train_tfidf)
corpus_train_SVD = SVD.transform(corpus_train_tfidf)
corpus_test_SVD = SVD.transform(corpus_test_tfidf)
corpus_train_SVD.shape

(7769, 100)

In [122]:
run_models(corpus_train_SVD, train_labels, corpus_test_SVD, test_labels, algos)

################## KNN5 #############
Aire sous la courbe: 63.775%

################## MLP #############
Aire sous la courbe: 65.137%

################## OVSR #############
Aire sous la courbe: 60.016%



In [45]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Concept #%d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


print_top_words(SVD, TFIDF.get_feature_names_out(), 10)

Concept #0: vs cts mln 000 loss net shr dlrs profit revs
Concept #1: said pct dlrs company billion bank shares mln lt stock
Concept #2: cts div qtly record april pay prior dividend sets march
Concept #3: billion bank pct stg mln february vs january money trade
Concept #4: loss 000 profit trade tonnes pct japan said dollar rate
Concept #5: 000 tonnes wheat said sugar net trade export vs ec
Concept #6: dlrs tonnes 1986 billion year quarter earnings 1987 january february
Concept #7: pct february january 000 shares stock rose stake rate common
Concept #8: stg tonnes 000 mln loss wheat bank money market pct
Concept #9: 000 dlrs bank billion oper fed money quarter share dollar
Concept #10: billion stock split dividend trade tonnes shares common board declared
Concept #11: split dividend stock share quarter earnings 1987 declared payable rate
Concept #12: oil billion crude gas split 000 opec barrels stock reserves
Concept #13: dlrs fed says mln shares offer rate pct week oper
Concept #14: tra

# 7. Word2Vec

Le but ici est de récupérer l'entièreté des mots du corpus et de les vectoriser. On va ensuite utiliser ces vecteurs pour entrainer un modèle de classification.
On pourra donc faire de la prédiction d'un mot à partir d'un contexte.

In [22]:
import gensim
import multiprocessing

cores = multiprocessing.cpu_count()
corpus = []
# Pour chaque document, on le tokenise (on récupère ses mots (pas les trop courts ou les trop longs)) et on le met dans le corpus
for i in range(len(train_documents)):
    corpus.append(gensim.utils.simple_preprocess(train_documents[i]))

7769

In [24]:
model_size = 100
model = gensim.models.Word2Vec(corpus, vector_size=model_size, sg=0, window=5, min_count=2, workers=cores - 1)

In [25]:
for i in range(100):
    model.train(corpus, total_examples=len(corpus), epochs=1)
    print('Train ', i)

Train  0
Train  1
Train  2
Train  3
Train  4
Train  5
Train  6
Train  7
Train  8
Train  9
Train  10
Train  11
Train  12
Train  13
Train  14
Train  15
Train  16
Train  17
Train  18
Train  19
Train  20
Train  21
Train  22
Train  23
Train  24
Train  25
Train  26
Train  27
Train  28
Train  29
Train  30
Train  31
Train  32
Train  33
Train  34
Train  35
Train  36
Train  37
Train  38
Train  39
Train  40
Train  41
Train  42
Train  43
Train  44
Train  45
Train  46
Train  47
Train  48
Train  49
Train  50
Train  51
Train  52
Train  53
Train  54
Train  55
Train  56
Train  57
Train  58
Train  59
Train  60
Train  61
Train  62
Train  63
Train  64
Train  65
Train  66
Train  67
Train  68
Train  69
Train  70
Train  71
Train  72
Train  73
Train  74
Train  75
Train  76
Train  77
Train  78
Train  79
Train  80
Train  81
Train  82
Train  83
Train  84
Train  85
Train  86
Train  87
Train  88
Train  89
Train  90
Train  91
Train  92
Train  93
Train  94
Train  95
Train  96
Train  97
Train  98
Train  99


In [27]:
model.save('./Word2vec_entraine.h5')

In [51]:
model.predict_output_word(['I','like'])

[('nations', 0.7614343),
 ('looks', 0.13128786),
 ('my', 0.041960564),
 ('countries', 0.015430394),
 ('ge', 0.008838033),
 ('sounds', 0.0050462154),
 ('areas', 0.0047625853),
 ('anything', 0.004585588),
 ('things', 0.0044911387),
 ('agreements', 0.0037674212)]

Exemple ci-dessus : on prédit le mot qui suit "I like" dans le corpus et il y a 76% de chance que ce soit "nations".

In [36]:
def word2vec_generator(texts, model, vector_size):
    dict_word2vec = {}
    # Pour chaque document, on récupère la moyenne des vecteurs de ses mots
    for index, word_list in enumerate(texts):
        arr = np.array([0.0 for i in range(0, vector_size)])
        nb_word = 0
        for word in word_list:
            try:
                arr += model[word]
                nb_word = nb_word + 1
            except KeyError:
                continue
        if (len(word_list) == 0):
            dict_word2vec[index] = arr
        else:
            dict_word2vec[index] = arr / nb_word
    df_word2vec = pd.DataFrame(dict_word2vec).T
    return df_word2vec

On va ici tokeniser les documents de train et de test

In [37]:
corpus_train_tokens = pd.Series(train_documents).apply(lambda document: gensim.utils.simple_preprocess(document))
corpus_test_tokens = pd.Series(test_documents).apply(lambda document: gensim.utils.simple_preprocess(document))

In [39]:
#On récupère le modèle entrainé qui prédit les mots à partir d'un contexte
model_wv_entraine = gensim.models.Word2Vec.load('./Word2vec_entraine.h5')
vector_size = model_wv_entraine.vector_size
corpus_train_wv_entraine = word2vec_generator(corpus_train_tokens, model_wv_entraine.wv, vector_size)
corpus_test_wv_entraine = word2vec_generator(corpus_test_tokens, model_wv_entraine.wv, vector_size)

In [41]:
corpus_train_wv_entraine.shape

(7769, 100)

In [70]:
run_models(corpus_train_wv_entraine, train_labels, corpus_test_wv_entraine, test_labels, algos)

################## RF #############
Cross-validation accuracy: 0.1904 (+/- 0.0221)

################## KNN #############
Cross-validation accuracy: 0.3432 (+/- 0.0705)

################## MLP #############
Cross-validation accuracy: 0.0094 (+/- 0.0082)

################## OVSR #############
Cross-validation accuracy: 0.7440 (+/- 0.0274)



L'OVR est le meilleur modèle. On va donc l'utiliser pour prédire les tags des documents de test.

In [42]:
ovr = OneVsRestClassifier(LinearSVC(random_state=0), n_jobs=-1)
ovr.fit(corpus_train_wv_entraine, train_labels)

OneVsRestClassifier(estimator=LinearSVC(random_state=0), n_jobs=-1)

On récupère les prédictions pour les documents de test
Pour la lecture, chaque ligne correspond à un article. Chaque colonne correspond à un tag. Si la valeur est à 1, c'est que l'article a ce tag.

In [73]:
p_test = ovr.predict(corpus_test_wv_entraine)
p_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

On va transformer les prédictions en tags à l'aide du MultiLabelBinarizer qui a servi à encoder les tags.

In [74]:
mlb.inverse_transform(p_test)

[('trade',),
 (),
 ('crude',),
 (),
 ('palm-oil', 'veg-oil'),
 ('ship',),
 (),
 ('grain', 'wheat'),
 ('gold',),
 (),
 (),
 ('interest', 'money-fx'),
 (),
 ('ipi',),
 ('trade',),
 ('earn',),
 ('earn',),
 ('money-fx',),
 ('bop', 'trade'),
 (),
 ('acq', 'gold'),
 ('ipi', 'jobs'),
 ('earn',),
 (),
 ('earn',),
 ('earn',),
 (),
 ('ship', 'trade'),
 (),
 ('sugar',),
 ('sugar',),
 ('acq',),
 ('money-fx',),
 ('crude',),
 ('oilseed', 'palm-oil', 'soybean'),
 ('acq', 'earn'),
 ('acq',),
 ('earn',),
 ('trade',),
 ('acq',),
 ('iron-steel',),
 ('earn',),
 ('ship', 'trade'),
 ('dlr', 'money-fx'),
 ('jobs',),
 ('money-fx',),
 ('acq',),
 ('sugar',),
 ('sugar',),
 ('earn',),
 (),
 ('earn',),
 ('money-fx',),
 ('acq',),
 ('earn',),
 ('earn',),
 ('acq',),
 ('acq',),
 ('acq',),
 ('cotton', 'livestock', 'trade'),
 ('earn',),
 ('ship',),
 ('earn',),
 ('ship',),
 ('earn',),
 ('earn',),
 ('earn',),
 ('interest', 'money-fx'),
 ('earn',),
 ('earn',),
 ('earn',),
 ('earn',),
 ('earn',),
 ('earn',),
 ('jobs', 'zinc

## Exemple pratique

In [67]:
def predict_article(article, model, mlb, ovr):
    token_article_test = gensim.utils.simple_preprocess(article)
    article_entraine = word2vec_generator([token_article_test], model.wv, vector_size)
    predict_article = ovr.predict(article_entraine)
    return mlb.inverse_transform(predict_article)

On va ici prédire les tags d'un article récupéré sur internet ici : https://www.centrenaturesante.com/article.php?p_ida=8
On le traduit en anglais et on le stocke dans un fichier texte.

In [72]:
article_test = open('./article.txt', 'r').read()
predict_article(article_test, model_wv_entraine, mlb, ovr)

[('grain', 'rice', 'tea', 'trade')]

On voit que le modèle prédit bien les tags de l'article.